## 1. Import Prerequisites

Run the following cell to confirm that the GPU is detected.

In [ ]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')


Found GPU at: /device:GPU:0


In order for torch to use the GPU, we need to identify and specify the GPU as the device.   
Later, in our training loop, we will load data onto the device. 


In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:
!pip install transformers

     |████████████████████████████████| 2.6 MB 7.1 MB/s 
     |████████████████████████████████| 3.3 MB 55.5 MB/s 
     |████████████████████████████████| 636 kB 62.1 MB/s 
     |████████████████████████████████| 895 kB 68.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


## 2. Parse


### 2.1 Load from file

In [65]:
# Check that dataset file is available
import glob

version_files = glob.glob('./versions_*.csv')

if len(version_files) == 0:
  raise SystemError("Cant find any versions!!")


# TODO: get latest uploaded version..
version_file = version_files[0]
print(f"Using - {version_file}")


Using - ./versions_2.csv


In [66]:
import pandas as pd
pd.options.display.max_colwidth = 10000


# Load the dataset into a pandas dataframe.
df = pd.read_csv(version_file)


# Report the number of sentences.
print(f'Number of sentences: {df.shape[0]}')

# Report the number of articles.
print(f'Number of unique articles: {df.article_id.unique().shape[0]}')
print(f'Number of articles with more than 2 versions: {df.loc[df.version > 1].article_id.unique().shape[0]}')
print(f'Number of articles with too many versions: {df.loc[df.version > 10].article_id.unique().shape[0]}')


Number of sentences: 187531
Number of unique articles: 28097
Number of articles with more than 2 versions: 11555
Number of articles with too many versions: 148


In [67]:
# Drop dups - 

print(f'Amount of lines before: {df.shape[0]}')
df.drop_duplicates(subset=["title"], keep='first', inplace=True)
print(f'Amount of lines after: {df.shape[0]}')

Amount of lines before: 187531
Amount of lines after: 39273


### 2.1 Problem finding in data

In [60]:
df.article_id.value_counts()

1.9792445                                   31
1.9802921                                   24
1.9796542                                   21
1.9808034                                   20
1.9781461                                   19
                                            ..
1.9419066                                    1
1.9567156                                    1
856681 at https://www.israelhayom.co.il      1
https://healthy.walla.co.il/item/3426351     1
860485 at https://www.israelhayom.co.il      1
Name: article_id, Length: 22254, dtype: int64

In [56]:
for problematic_article in df.loc[df.version > 15].article_id.unique():
  print(f"============== start: {problematic_article} =================")

  test_case = df.loc[df.article_id == "1.9792445"]

  test_case.sort_values(by=["version"], inplace=True)
  # test_case[["title", "version", "article_id"]]

  for _, r in test_case.iterrows():
    print(r.title)
  
  print(f"-------------- end {problematic_article} -----------------")

============== start: 1.9514062 =================
מטחים כבדים על אשקלון ועוטף עזה; היערכות לסבב של כמה ימי לחימה
מטחים כבדים לאשקלון; שני מבנים בעיר נפגעו ושלושה אנשים נפצעו
בתים באשקלון ספגו פגיעות ישירות; שבעה נפצעו, שניים בינוני
בתים באשקלון ספגו פגיעות ישירות; שבעה נפצעו, שניים במצב בינוני
בתים באשקלון ספגו פגיעות ישירות; שבעה נפצעו, בהם ארבעה בני משפחה אחת
אושר גיוס מילואים; דיווח בעזה: פעילים בג'יהאד נהרגו בתקיפת צה"ל
כוכבי: נערכים למבצע ללא מגבלת זמן; דיווח בעזה: פעילי ג'יהאד נהרגו בתקיפת צה"ל
כוכבי: נערכים למבצע ללא מגבלת זמן; דיווח בעזה: בכירי ג'יהאד נהרגו בתקיפת צה"ל
כוכבי: נערכים למבצע ללא מגבלת זמן; חמאס: הירי בהמשך יהיה עוצמתי יותר
רקטות פגעו באשדוד ובאשקלון; חמאס: הירי בהמשך יהיה עוצמתי יותר
פגיעות ישירות במבנים באשדוד ובאשקלון; חמאס: הירי בהמשך יהיה עוצמתי יותר
דובר צה"ל: תושבי אשקלון מתבקשים להישאר במרחב המוגן עד להודעה חדשה
שני פצועים בפגיעה ישירה במבנה באשקלון, אחת מהם במצב קשה
שתי הרוגות בפגיעות ישירות במבנים באשקלון, פצוע נוסף במצב קשה
שתי הרוגות בפגיעות ישירות במבנ

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


התחדש הירי לעבר המרכז, השפלה והדרום; מצב חירום בלוד בעקבות ההתפרעויות
התחדש הירי לעבר גוש דן והדרום; מצב חירום בלוד בעקבות ההתפרעויות
מטחים כבדים מהדרום ועד השרון; מצב חירום בלוד בעקבות ההתפרעויות
ילד ואישה נהרגו מפגיעת רקטה בלוד; מטחים כבדים מהדרום ועד השרון
ילדה ואישה נהרגו מפגיעת רקטה בלוד; מטחים כבדים מהדרום ועד השרון
ילדה וגבר נהרגו מפגיעת רקטה בלוד; מטחים כבדים מהדרום ועד השרון
אב ובתו נהרגו מפגיעת רקטה סמוך ללוד; מטחים כבדים מהדרום ועד השרון
-------------- end 1.9802921 -----------------
============== start: https://news.walla.co.il/item/3435317 =================
מטחים כבדים על אשקלון ועוטף עזה; היערכות לסבב של כמה ימי לחימה
מטחים כבדים לאשקלון; שני מבנים בעיר נפגעו ושלושה אנשים נפצעו
בתים באשקלון ספגו פגיעות ישירות; שבעה נפצעו, שניים בינוני
בתים באשקלון ספגו פגיעות ישירות; שבעה נפצעו, שניים במצב בינוני
בתים באשקלון ספגו פגיעות ישירות; שבעה נפצעו, בהם ארבעה בני משפחה אחת
אושר גיוס מילואים; דיווח בעזה: פעילים בג'יהאד נהרגו בתקיפת צה"ל
כוכבי: נערכים למבצע ללא מגבלת זמן; דיווח בעז

In [57]:


# Display 10 random rows from the data.
# df.loc[(df.article_id == "1.9812346")]
df.sort_values(by=["article_id"], inplace=True)

# df[["title", "version", "article_id"]].head(20)
# df.loc[df.version > 100][["title", "version", "article_id"]].article_id


df[["title", "version", "article_id"]].head(10)


# df.sample(10)


,title,version,article_id
37201,"סלט ירוק עם משמשים, גבינת עזים ואגוזים",1.0,1.1022818
7881,"בקלנועית, בדרך לקרוסטיני",1.0,1.1048736
8043,מתכונים לקרוסטיני,2.0,1.1048736
5882,קומקוואט: תפוז בזעיר אנפין,1.0,1.1067876
40473,מסע הצלב של אורי-בורי,1.0,1.1096986
20842,למצות את הפוטנציאל: שלושה מתכונים מפתיעים עם מצות,1.0,1.1098187
24890,הגרסה של אייל שני למצה בריי,1.0,1.1098211
7782,בראוניז מושלמים ב-32 דקות,1.0,1.1125704
7771,העמוד האחורי,1.0,1.1164652
30370,סיור בירושלים שאתם לא מכירים,1.0,1.1167366


### 2.3 How many lines have more than one version ?

#### 2.3.1 Scan versions

In [68]:
v = df.article_id.value_counts()

for i in range(1, v.max() + 1):
  matching_articles = list(v[v == i].keys())
  print(f"Article ids with {i} versions, amount = {len(matching_articles)} \nids - {matching_articles}\n\n")


Article ids with 1 versions, amount = 19928 
ids - ['847181 at https://www.israelhayom.co.il', '1.9689565', '876139 at https://www.israelhayom.co.il', '1.9821989', '1.9574191', '1.9584671', '861575 at https://www.israelhayom.co.il', '872403 at https://www.israelhayom.co.il', '849697 at https://www.israelhayom.co.il', 'https://news.walla.co.il/item/3420944', '850851 at https://www.israelhayom.co.il', '871197 at https://www.israelhayom.co.il', '852919 at https://www.israelhayom.co.il', 'https://news.walla.co.il/item/3434701', '1.9598449', '1.9648926', '1.9410281', '1.9796634', 'https://news.walla.co.il/item/3422063', '849945 at https://www.israelhayom.co.il', '874299 at https://www.israelhayom.co.il', '1.9818153', '1.9797222', '1.9806424', '850951 at https://www.israelhayom.co.il', '1.7257025', '846167 at https://www.israelhayom.co.il', '1.9941200', '1.9768553', '854999 at https://www.israelhayom.co.il', '871919 at https://www.israelhayom.co.il', 'https://news.walla.co.il/item/3416022', 

#### 2.3.2 Sample an Example

In [69]:
random_article = v[v == 18].sample()

# random_article = v[v == 10]


random_article_ids = list(random_article.to_dict().keys())
print(f"Article id - {random_article_ids}")

df[df.article_id.isin(random_article_ids)].sort_values(inplace=False, by=["article_id", "version"])[["title", "version", "article_id"]]


Article id - ['1.9900111']


,title,version,article_id
137740,"ח""כ אלחרומי מרע""ם מאיים לא לתמוך בממשלת בנט-לפיד",1,1.9900111
138074,"לאחר שאיים לא לתמוך, ח""כ אלחרומי: תהיה היום ממשלה",2,1.9900111
138241,שעתיים לפני השבעת הממשלה: ראשי סיעות הקואליציה מתכנסים לישיבה בכנסת,3,1.9900111
138285,שעתיים להשבעת הממשלה: ראשי סיעות הקואליציה החדשה מתכנסים לישיבה בכנסת,4,1.9900111
138584,ראשי סיעות הקואליציה החדשה נועדו בכנסת,5,1.9900111
138760,ישיבת השבעת הממשלה צפויה להתחיל ב-16:00; ראשי הקואליציה נועדו בכנסת,6,1.9900111
139088,החלה הישיבה המיוחדת להשבעת הממשלה ה-36: בנט ינאם מיד,7,1.9900111
139138,"ראש הממשלה המיועד בנט נואם כעת במליאה, לקראת השבעת הממשלה ה-36",8,1.9900111
139176,"בנט בנאום במליאה: מודה לנתניהו על שירות ארוך שנים, כעת ההנהגה עוברת לדור הבא",9,1.9900111
139350,"בנט: בלי לפיד לא היינו כאן היום, נתניהו סלל את הדרך לצירוף רע""ם",10,1.9900111


## Versions side by side 

#### First option - for each article all versions in a row

In [226]:
MAX_VERSIONS = df.article_id.value_counts().max()

# cols = ["article_id"] #+ [f"version_{i}" for i in range(1, MAX_VERSIONS + 1)]

# articles_df = pd.DataFrame()

# Insert all the article ids
# articles_df["article_id"] = df.article_id.unique()

# articles_df[["article_id", "version_1"]] = df.loc[df.version == 1][["article_id", "title"]]

# test_version = df.loc[df.version == 1].sample()
# for _, row in test_version.iterrows():
#   test_version_article_id = row.article_id
#   test_version_title = row.title

# print(f"test - id - {test_version_article_id}, \n title-{test_version_title}\n----")
# articles_df[articles_df.article_id == test_version_article_id]

def f(r):
  # article_id = r.sample().article_id.iat[0]
  # y = r[["title", "version"]].set_index("version").T
  y = pd.DataFrame()
  y["article_id"] = [r.sample().article_id.iat[0]]

  return y

articles_df = df.groupby(["article_id"]).apply(f)

In [227]:
articles_df.reset_index

# a = articles_df.reset_index()
# a.head()

,,article_id
article_id,,
1.1048736,0,1.1048736
1.1108024,0,1.1108024
1.1241444,0,1.1241444
1.1421550,0,1.1421550
1.1438126,0,1.1438126
...,...,...
https://travel.walla.co.il/item/3428023,0,https://travel.walla.co.il/item/3428023
https://travel.walla.co.il/item/3430450,0,https://travel.walla.co.il/item/3430450
https://travel.walla.co.il/item/3434914,0,https://travel.walla.co.il/item/3434914


#### Second option - sequential versions in a row

### Drop single version articles

In [70]:
single_version_article_ids = list(v[v == 1].to_dict().keys())

lines_to_be_removed = df[df.article_id.isin(single_version_article_ids)][["version", "title", "article_id"]]

print(f"Removing {lines_to_be_removed.size} lines!")

# Uncomment the next line to drop those articles
# df.drop(df[df.article_id.isin(single_version_article_ids)].index, inplace=True)

print(f'Number of unique articles: {df.article_id.unique().shape[0]}')

Removing 59784 lines!
Number of unique articles: 7920


### 2.3 Almost done paring data

In [ ]:
MAX_WORDS = df.amount_of_words.max()
print(f"Max words in sentence is - {MAX_WORDS}")


Max words in sentence is - 24.0


Let's extract the sentences and labels of our training set as numpy ndarrays.

In [ ]:
sentences = df.title.values
labels = df.label.values

# 3. Tokenization & Input Formatting


## 3.1. BERT Tokenizer

In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM


# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = AutoTokenizer.from_pretrained("avichr/heBERT")
# tokenizer = AutoTokenizer.from_pretrained("onlplab/alephbert-base")


Loading BERT tokenizer...


Let's apply the tokenizer to one sentence just to see the output.

In [ ]:
import random

example_sentence = sentences[random.choice(range(len(sentences)))]
# Print the original sentence.
print(' Original: ', example_sentence)

# Print the sentence split into tokens.
print('Tokenized: ', tokenizer.tokenize(example_sentence))

# Print the sentence mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(example_sentence)))


 Original:  הציפייה לברברים היא הברבריות עצמה
Tokenized:  ['הצי', '##פייה', 'לבר', '##ברים', 'היא', 'הבר', '##בריו', '##ת', 'עצמה']
Token IDs:  [6033, 4281, 3444, 1843, 1667, 3464, 6576, 1019, 3523]


In [ ]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []

# For every sentence...
for sent in sentences:
    # `encode` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    encoded_sent = tokenizer.encode(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'

                        # This function also supports truncation and conversion
                        # to pytorch tensors, but we need to do padding, so we
                        # can't use these features :( .
                        #max_length = 128,          # Truncate all sentences.
                        #return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.
    input_ids.append(encoded_sent)

# Print sentence 0, now as a list of IDs.
print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])

TypeError: ignored

In [ ]:
# We'll borrow the `pad_sequences` utility function to do this.
from keras.preprocessing.sequence import pad_sequences

# Set the maximum sequence length.
MAX_LEN = 32

print('\nPadding/truncating all sentences to %d values...' % MAX_LEN)

print('\nPadding token: "{:}", ID: {:}'.format(tokenizer.pad_token, tokenizer.pad_token_id))

# Pad our input tokens with value 0.
# "post" indicates that we want to pad and truncate at the end of the sequence,
# as opposed to the beginning.
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", 
                          value=0, truncating="post", padding="post")

print('\nDone.')


Padding/truncating all sentences to 32 values...

Padding token: "[PAD]", ID: 0

Done.


In [ ]:
# Create attention masks
attention_masks = []

# For each sentence...
for sent in input_ids:
    
    # Create the attention mask.
    #   - If a token ID is 0, then it's padding, set the mask to 0.
    #   - If a token ID is > 0, then it's a real token, set the mask to 1.
    att_mask = [int(token_id > 0) for token_id in sent]
    
    # Store the attention mask for this sentence.
    attention_masks.append(att_mask)

In [ ]:
# Use train_test_split to split our data into train and validation sets for
# training
from sklearn.model_selection import train_test_split

# Use 90% for training and 10% for validation.
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, 
                                                            random_state=2018, test_size=0.1)
# Do the same for the masks.
train_masks, validation_masks, _, _ = train_test_split(attention_masks, labels,
                                             random_state=2018, test_size=0.1)

In [ ]:
# Convert all inputs and labels into torch tensors, the required datatype 
# for our model.
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)

train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)

train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here.
# For fine-tuning BERT on a specific task, the authors recommend a batch size of
# 16 or 32.

batch_size = 32

# Create the DataLoader for our training set.
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# Create the DataLoader for our validation set.
validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)


##????

In [ ]:
model = AutoModelForMaskedLM.from_pretrained("avichr/heBERT")

from transformers import pipeline
fill_mask = pipeline(
    "fill-mask",
    model="avichr/heBERT",
    tokenizer="avichr/heBERT"
)

options = fill_mask("הקורונה לקחה את [MASK] ולנו לא נשאר דבר.")

options

[{'score': 0.14047907292842865,
  'sequence': 'הקורונה לקחה את הילדים ולנו לא נשאר דבר.',
  'token': 3096,
  'token_str': 'הילדים'},
 {'score': 0.04530879110097885,
  'sequence': 'הקורונה לקחה את הכסף ולנו לא נשאר דבר.',
  'token': 5289,
  'token_str': 'הכסף'},
 {'score': 0.0362359881401062,
  'sequence': 'הקורונה לקחה את הכלב ולנו לא נשאר דבר.',
  'token': 12737,
  'token_str': 'הכלב'},
 {'score': 0.035021472722291946,
  'sequence': 'הקורונה לקחה את הילדה ולנו לא נשאר דבר.',
  'token': 12178,
  'token_str': 'הילדה'},
 {'score': 0.02997061051428318,
  'sequence': 'הקורונה לקחה את הרכב ולנו לא נשאר דבר.',
  'token': 3806,
  'token_str': 'הרכב'}]

In [ ]:
max(options, key=lambda x: x["score"])

{'score': 0.14047907292842865,
 'sequence': 'הקורונה לקחה את הילדים ולנו לא נשאר דבר.',
 'token': 3096,
 'token_str': 'הילדים'}